In [11]:
import numpy as np
from numpy import vstack
from numpy import argmax
from sklearn.gaussian_process import GaussianProcessRegressor
from warnings import catch_warnings
from warnings import simplefilter
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import norm

# objective function
def objective(x1, x2, x3, noise=0.1):
    noise = np.random.normal(loc=0, scale=noise)
    # result = ((x1 * 10e+0) / (x2 * x3))
    result  = x1/10 + 1/x2 + 1/x3
    if (x2==0 or x3==0):
      return 0 + noise
    return result + noise

# surrogate or approximation for the objective function
def surrogate(model, X):
    # catch any warning generated when making a prediction
    with catch_warnings():
        # ignore generated warnings
        simplefilter("ignore")
        return model.predict(X, return_std=True)

# probability of improvement acquisition function
def acquisition(X, Xsamples, model):
    # calculate the best surrogate score found so far
    yhat, _ = surrogate(model, X)
    best = max(yhat)
    # calculate mean and stdev via surrogate function
    mu, std = surrogate(model, Xsamples)
    # calculate the probability of improvement
    probs = norm.cdf((mu - best) / (std+1E-9))
    return probs

# optimize the acquisition function
def opt_acquisition(X, y, model):
    # random search, generate random samples
    Xsamples = np.zeros((100, 3))
    Xsamples[:, 0] = np.random.uniform(0, 20, size=(100,))
    Xsamples[:, 1] = np.random.uniform(0, 20, size=(100,))
    Xsamples[:, 2] = np.random.uniform(0, 13, size=(100,))
    # calculate the acquisition function for each sample
    scores = acquisition(X, Xsamples, model)
    # locate the index of the largest scores
    ix = argmax(scores)
    return Xsamples[ix, :]

# # plot real observations vs surrogate function
# def plot(X, y, model):
#     fig = plt.figure()
#     ax = fig.add_subplot(111, projection='3d')
#     ax.scatter(X[:, 0], X[:, 1], y)
#     Xsamples = np.meshgrid(np.arange(0, 100, 10), np.arange(0, 100, 10), np.arange(1, 13, 1))
#     Xsamples = np.vstack([Xsamples[0].flatten(), Xsamples[1].flatten(), Xsamples[2].flatten()]).T
#     ysamples, _ = surrogate(model, Xsamples)
#     ax.plot_surface(Xsamples[:, 0].reshape((10, 10, 12)), Xsamples[:, 1].reshape((10, 10, 12)), Xsamples[:, 2].reshape((10, 10, 12)), ysamples.reshape((10, 10, 12)))
#     plt.show()

# input data
X = np.array([[2, 5, 13], [20, 15, 6.5], [2, 5, 6.5], [2, 15, 13], [10, 5, 13], [10, 15, 6.5], [10, 5, 6.5], [10, 15, 13], [20, 5, 13], [2, 15, 6.5], [20, 5, 6.5], [20, 15, 13], [0, 0, 0]])
# y = np.array([20.9, 27.4, 29.7, 34.2, 30.1, 29.6, 41.7, 21.8, 28.1, 42, 32.2, 53.8, 18.7]).reshape(-1, 1)
y = np.array([0.11, 0.21, 0.57, 0.62, 0.58, 0.54, 0.85, 0.14, 0.41, 0.9, 0.71, 0.95, 0.1]).reshape(-1, 1)

# define the model
model = GaussianProcessRegressor()

# fit the model
model.fit(X, y)

# # plot beforehand
# plot(X, y, model)

# perform the optimization process
for i in range(11):
    # select the next point to sample
    x = opt_acquisition(X, y, model)
    # sample the point
    actual = objective(x[0], x[1], x[2])
    # summarize the finding
    est, _ = surrogate(model, [x])
    # print(est[0])
    print('>inumber = %.0f, x1=%.3f, x2=%.3f, x3=%.3f, f()=%.10f (eval on surrogate fn at the point), actual=%.10f (eval on defined objective function)' % (i, x[0], x[1], x[2], est, actual))
    # add the data to the dataset
    X = vstack((X, x))
    y = vstack((y, [[actual]]))
    # update the model
    model.fit(X, y)

# # plot all samples and the final surrogate function
# plot(X, y, model)

# best result

ix = argmax(y)
print('It is expected that as we move closer to the optimal, difference between f() and actual should decrease')
print('Best Result: x1=%.3f, x2=%.3f, x3=%.3f, f()=%.10f' % (X[ix, 0], X[ix, 1], X[ix, 2], y[ix]))


>inumber = 0, x1=18.928, x2=15.506, x3=12.711, f()=0.4514442727 (eval on surrogate fn at the point), actual=2.0396660309 (eval on defined objective function)
>inumber = 1, x1=11.050, x2=4.449, x3=6.986, f()=0.3736274113 (eval on surrogate fn at the point), actual=1.3407545260 (eval on defined objective function)
>inumber = 2, x1=19.762, x2=15.066, x3=12.058, f()=1.0484554081 (eval on surrogate fn at the point), actual=2.3249623151 (eval on defined objective function)
>inumber = 3, x1=18.118, x2=16.841, x3=12.255, f()=0.4628749933 (eval on surrogate fn at the point), actual=2.0992901627 (eval on defined objective function)
>inumber = 4, x1=1.097, x2=14.313, x3=6.602, f()=0.4703658659 (eval on surrogate fn at the point), actual=0.3514526190 (eval on defined objective function)
>inumber = 5, x1=12.009, x2=5.202, x3=7.625, f()=0.4870520545 (eval on surrogate fn at the point), actual=1.5550237403 (eval on defined objective function)
>inumber = 6, x1=19.766, x2=14.041, x3=10.253, f()=0.27963